In [ ]:
from gensim.test.utils import common_texts
from gensim.models import Word2Vec
import nltk
import json
VECTOR_SIZE=50
WINDOW_SIZE=5
THREADS=4
CUTOFF_FREQ=1
EPOCHS=100

def list_of_lists(sentences):
    tokenized_sentences=[]
    for sentence in sentences:
        tokenized_sentences.append(nltk.word_tokenize(sentence))
    return tokenized_sentences

def train_gensim_model(corpus):
    tokenized_sentences=list_of_lists(corpus)
    model = Word2Vec(sentences=tokenized_sentences, vector_size=VECTOR_SIZE, window=WINDOW_SIZE, min_count=CUTOFF_FREQ, workers=THREADS)
    model.build_vocab(tokenized_sentences)
    model.train(corpus_iterable=tokenized_sentences,total_examples=model.corpus_count,epochs=EPOCHS)
    print(model.corpus_count)
    return model

def read_data(path):
    with open(path,'r') as file:
        data = json.load(file)
    return data

def fix_json_file(path):
    fixed_file=open("fixed_"+path,"a")
    fixed_file.write("[\n")
    with open(path,'r') as file:
        for line in file:
            fixed_file.write(line[:-1]+",\n")
    fixed_file.seek(fixed_file.tell()-3)
    fixed_file.truncate()
    fixed_file.write("]")
    fixed_file.close()
        

In [ ]:
data=read_data("D:/CMP/4/1/NLP/Project/data/fixed_PIZZA_dev.json")
corpus=[]
for d in data:
    corpus.append(d["dev.SRC"])
model=train_gensim_model(corpus)


In [ ]:
print(model.wv.key_to_index.keys())

In [ ]:
tokenized_sentences=list_of_lists(corpus)
for sentence in tokenized_sentences:
    for word in sentence:
        print(word,model.wv.most_similar(word))

In [ ]:
import gensim.downloader
glove_vectors = gensim.downloader.load('glove-twitter-25')

glove_vectors.most_similar('twitter')


In [ ]:
glove_vectors = gensim.downloader.load('glove-wiki-gigaword-50')
glove_vectors.most_similar('very')


In [ ]:
glove_vectors.vector_size

In [ ]:
from transformers import BertTokenizer, BertModel
import pandas as pd
import numpy as np
import torch

In [ ]:
model = BertModel.from_pretrained("bert-base-uncased",output_hidden_states = True,)
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
def bert_text_preparation(text, tokenizer):
    marked_text = "[CLS] " + text + " [SEP]"
    tokenized_text = tokenizer.tokenize(marked_text)
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
    segments_ids = [1]*len(indexed_tokens)
    tokens_tensor = torch.tensor([indexed_tokens])
    segments_tensors = torch.tensor([segments_ids])

    return tokenized_text, tokens_tensor, segments_tensors

def get_bert_embeddings(tokens_tensor, segments_tensors, model):
    with torch.no_grad():
        outputs = model(tokens_tensor, segments_tensors)
        hidden_states = outputs[2][1:]
    token_embeddings = hidden_states[-1]
    token_embeddings = torch.squeeze(token_embeddings, dim=0)
    list_token_embeddings = [token_embed.tolist() for token_embed in token_embeddings]

    return list_token_embeddings



In [ ]:
texts = ["bank",
        "The river bank was flooded.",
        "The bank vault was robust.",
        "He had to bank on her for support.",
        "The bank was out of money.",
        "The bank teller was a man."]
target_word_embeddings = []

for text in texts:
    tokenized_text, tokens_tensor, segments_tensors = bert_text_preparation(text, tokenizer)
    list_token_embeddings = get_bert_embeddings(tokens_tensor, segments_tensors, model)
    
    word_index = tokenized_text.index('bank')
    word_embedding = list_token_embeddings[word_index]

    target_word_embeddings.append(word_embedding)


In [ ]:
print(len(target_word_embeddings))